###### 1.Nous avions fait regression OLS en utilisant les 4096 features extraites de la prédiction des classes d'intensité lumineuses nocturnes pour les 1084 ménages (en ensemble d'entrainement) avec le logarithme de la consommation par tete
###### 2.Nous avions obtenu un R^2 de 58.4%  dans l'ensemble d'entrainement et une prédiction de la pauvreté de 29.43% pour cet ensemble d'entrainement
###### 3. Il s'agit ici d'utiliser les nos prédictions pour estimer la pauvreté dans la zone urbaine et estimer la pauvreté en zone rurale en se basant sur le modèle OLS des 1084 ménages
###### 4. Ces prédictions seront comparés à celle que nous avons obtenues en divisant toute la base des 12997 ménages en fonction du milieu rurale ou urbain et nous avons estimer la pauvreté en rural en utilisant toutes entrées de nos données d'enquetes qui provenaient de la zone rurale et nous avons fait autant pour la zone urbaine. 

In [1]:
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib
import pickle
import os
import statsmodels.api as sm

In [2]:
import pandas as pd
import pickle
import os
import statsmodels.api as sm

# Définir les chemins des fichiers
BASE_DIR = r'E:\29 Aout 2024'
data_civ_path = os.path.join(BASE_DIR, 'DataCIV2.csv')
df_consumption_train_path = os.path.join(BASE_DIR, 'df_consumption_train_50m_moitie_IPM_100.csv')

# Lire les fichiers CSV
data_civ = pd.read_csv(data_civ_path)
df = pd.read_csv(df_consumption_train_path)


##### Traitement des données et organisations des données dans un dataframe afin qu'elles soient aisément exploitable

In [3]:
# Renommer les colonnes 'GPS__Latitude' et 'GPS__Longitude' dans data_civ
data_civ_renamed = data_civ.rename(columns={'GPS__Latitude': 'menage_lat', 'GPS__Longitude': 'menage_lon'})

# Fusionner les DataFrames sur les colonnes 'menage_lat' et 'menage_lon'
merged_df = pd.merge(df, data_civ_renamed[['menage_lat', 'menage_lon', 'grappe', 'region', 'milieu', 'hhweight', 'hhsize']],
                     on=['menage_lat', 'menage_lon'], how='left')

# Choisir la colonne 'grappe' à conserver (par exemple, on garde 'grappe_x' qui vient de 'df')
merged_df = merged_df.rename(columns={'grappe_x': 'grappe'}).drop(columns=['grappe_y'])

# Sélectionner les colonnes à conserver
columns_to_keep = ['image_name', 'menage_lat', 'menage_lon','hhweight', 'hhsize', 'pcexp',
    'is_train', 'nightlights_50m','grappe', 'region', 'milieu', 
    'nightlights_50m_bins', 'feat_index'
] + [f'feature_{i}' for i in range(4096)]


In [4]:
merged_df

,image_name,menage_lat,menage_lon,grappe,is_train,pcexp,nightlights_50m,nightlights_50m_bins,feat_index,feature_0,...,feature_4090,feature_4091,feature_4092,feature_4093,feature_4094,feature_4095,region,milieu,hhweight,hhsize
0,10.0110053_-5.9028431_1.png,10.011005,-5.902843,167,True,247799.03,3.541645,3,48300,-1.002076,...,-0.851106,-0.577896,1.326019,-0.120750,-0.162769,-0.923936,PORO,Urbain,547.83280,6
1,10.0110053_-5.9028431_10.png,10.011005,-5.902843,167,True,247799.03,3.541645,3,48301,-0.980855,...,-1.212620,-0.569172,0.923374,-0.261559,-0.590774,-1.391140,PORO,Urbain,547.83280,6
2,10.0110053_-5.9028431_100.png,10.011005,-5.902843,167,True,247799.03,3.541645,3,48302,-0.986348,...,-1.305606,-0.245702,1.048913,-0.314233,-0.292801,-0.684724,PORO,Urbain,547.83280,6
3,10.0110053_-5.9028431_11.png,10.011005,-5.902843,167,True,247799.03,3.541645,3,48303,-1.643712,...,-1.455011,-0.801300,1.673155,0.065971,-0.225988,-1.200723,PORO,Urbain,547.83280,6
4,10.0110053_-5.9028431_12.png,10.011005,-5.902843,167,True,247799.03,3.541645,3,48304,-1.378967,...,-1.619301,-0.555699,1.143866,0.019219,0.029121,-1.007114,PORO,Urbain,547.83280,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108395,9.9984151404351_-5.579286608845_95.png,9.998415,-5.579287,1047,True,593999.44,2.232362,4,108395,-0.033117,...,-0.470930,1.747882,-0.184641,-0.930760,-0.928856,-2.059495,TCHOLOGO,Rural,268.36096,3
108396,9.9984151404351_-5.579286608845_96.png,9.998415,-5.579287,1047,True,593999.44,2.232362,4,108396,0.101507,...,-1.206484,1.420752,-0.671785,0.271795,-0.150112,-2.400492,TCHOLOGO,Rural,268.36096,3
108397,9.9984151404351_-5.579286608845_97.png,9.998415,-5.579287,1047,True,593999.44,2.232362,4,108397,0.451813,...,-0.612537,-0.102968,-1.036566,-0.540336,-0.244181,-2.330938,TCHOLOGO,Rural,268.36096,3
108398,9.9984151404351_-5.579286608845_98.png,9.998415,-5.579287,1047,True,593999.44,2.232362,4,108398,0.339873,...,-0.690120,1.197837,-0.844013,-0.336367,-0.098016,-2.119674,TCHOLOGO,Rural,268.36096,3


In [5]:
# Créer le DataFrame final avec les colonnes sélectionnées
final_df = merged_df[columns_to_keep]

In [6]:
final_df

,image_name,menage_lat,menage_lon,hhweight,hhsize,pcexp,is_train,nightlights_50m,grappe,region,...,feature_4086,feature_4087,feature_4088,feature_4089,feature_4090,feature_4091,feature_4092,feature_4093,feature_4094,feature_4095
0,10.0110053_-5.9028431_1.png,10.011005,-5.902843,547.83280,6,247799.03,True,3.541645,167,PORO,...,-0.265637,0.261541,-2.311038,0.971401,-0.851106,-0.577896,1.326019,-0.120750,-0.162769,-0.923936
1,10.0110053_-5.9028431_10.png,10.011005,-5.902843,547.83280,6,247799.03,True,3.541645,167,PORO,...,-0.389408,1.504738,-1.186777,0.620498,-1.212620,-0.569172,0.923374,-0.261559,-0.590774,-1.391140
2,10.0110053_-5.9028431_100.png,10.011005,-5.902843,547.83280,6,247799.03,True,3.541645,167,PORO,...,-0.353826,1.331153,-1.689120,1.113854,-1.305606,-0.245702,1.048913,-0.314233,-0.292801,-0.684724
3,10.0110053_-5.9028431_11.png,10.011005,-5.902843,547.83280,6,247799.03,True,3.541645,167,PORO,...,-0.543742,1.001645,-1.674773,1.178820,-1.455011,-0.801300,1.673155,0.065971,-0.225988,-1.200723
4,10.0110053_-5.9028431_12.png,10.011005,-5.902843,547.83280,6,247799.03,True,3.541645,167,PORO,...,-0.316980,0.953547,-1.627375,0.955977,-1.619301,-0.555699,1.143866,0.019219,0.029121,-1.007114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108395,9.9984151404351_-5.579286608845_95.png,9.998415,-5.579287,268.36096,3,593999.44,True,2.232362,1047,TCHOLOGO,...,-0.376207,-0.748926,-0.025150,0.223708,-0.470930,1.747882,-0.184641,-0.930760,-0.928856,-2.059495
108396,9.9984151404351_-5.579286608845_96.png,9.998415,-5.579287,268.36096,3,593999.44,True,2.232362,1047,TCHOLOGO,...,-0.258317,-1.068266,-1.064823,-0.377628,-1.206484,1.420752,-0.671785,0.271795,-0.150112,-2.400492
108397,9.9984151404351_-5.579286608845_97.png,9.998415,-5.579287,268.36096,3,593999.44,True,2.232362,1047,TCHOLOGO,...,-0.543727,-0.755949,-0.123490,-0.680691,-0.612537,-0.102968,-1.036566,-0.540336,-0.244181,-2.330938
108398,9.9984151404351_-5.579286608845_98.png,9.998415,-5.579287,268.36096,3,593999.44,True,2.232362,1047,TCHOLOGO,...,-0.510764,-0.729567,-0.648976,-0.596446,-0.690120,1.197837,-0.844013,-0.336367,-0.098016,-2.119674


In [7]:
# Sauvegarder le DataFrame final
output_path = os.path.join(BASE_DIR, 'merged_data.csv')
final_df.to_csv(output_path, index=False)

print(f"Le fichier fusionné a été sauvegardé sous : {output_path}")

Le fichier fusionné a été sauvegardé sous : E:\29 Aout 2024\merged_data.csv


In [9]:
final_df['milieu']

0         Urbain
1         Urbain
2         Urbain
3         Urbain
4         Urbain
           ...  
108395     Rural
108396     Rural
108397     Rural
108398     Rural
108399     Rural
Name: milieu, Length: 108400, dtype: object

In [10]:
# Renommer final_df en train_1084
train_1084 = final_df


In [11]:
# Diviser train_1084 en fonction de la colonne 'milieu'
train_1084_urbain = train_1084[train_1084['milieu'] == 'Urbain']
train_1084_rural = train_1084[train_1084['milieu'] == 'Rural']

# Vérifier les tailles des DataFrames
print(f"Taille de train_1084_urbain : {train_1084_urbain.shape}")
print(f"Taille de train_1084_rural : {train_1084_rural.shape}")


Taille de train_1084_urbain : (44000, 4109)
Taille de train_1084_rural : (64400, 4109)


In [19]:
import numpy as np
import statsmodels.api as sm

# Sélectionner les caractéristiques (features) X_train_1084 et la variable cible y
X_train_1084 = train_1084.iloc[:, 13:]  # Caractéristiques à partir de la colonne 13
y = train_1084['pcexp']  # Variable cible

# Prendre le logarithme de y
y_log = np.log(y)

# Ajouter une constante (intercept) à X_train_1084 pour le modèle OLS
X_train_1084 = sm.add_constant(X_train_1084)

# Effectuer la régression OLS
model = sm.OLS(y_log, X_train_1084)
results = model.fit()

# Afficher le résumé des résultats
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  pcexp   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     36.00
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        15:53:00   Log-Likelihood:                -59397.
No. Observations:              108400   AIC:                         1.269e+05
Df Residuals:                  104336   BIC:                         1.659e+05
Df Model:                        4063                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -2968.9277   2519.991     -1.178   

In [30]:
# Il faut remarquer que le script nous met en garde au sujet de la multicolinéarité

In [20]:
# Chemin du dossier de destination
save_dir = r'E:\29 Aout 2024'

In [21]:
# Définir le chemin complet pour sauvegarder le modèle
model_filename = os.path.join(save_dir, 'ols_model_train_1084.pkl')

# Sauvegarder le modèle
with open(model_filename, 'wb') as file:
    pickle.dump(results, file)

print(f"Le modèle OLS a été sauvegardé sous le nom : {model_filename}")

Le modèle OLS a été sauvegardé sous le nom : E:\29 Aout 2024\ols_model_train_1084.pkl


In [22]:
'''
#Pour charger le modèle
import pickle

# Chemin du fichier où le modèle est sauvegardé
model_filename = r'E:\29 Aout 2024\ols_model_train_1084.pkl'

# Chargement du modèle sauvegardé
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

# Utilisation du modèle chargé pour faire des prédictions ou d'autres analyses
print(loaded_model.summary())

# Supposons que nous avons un nouveau jeu de données X_new pour lequel on souhaite prédire y_log
X_new = sm.add_constant(X_new)  # Ajout de l'intercept si

# Faire des prédictions
y_pred_log = loaded_model.predict(X_new)

# Si vous souhaitez obtenir les prédictions en échelle normale (non log), vous pouvez utiliser np.exp
y_pred = np.exp(y_pred_log)

'''

"\n#Pour charger le modèle\nimport pickle\n\n# Chemin du fichier où le modèle est sauvegardé\nmodel_filename = r'E:\x029 Aout 2024\\ols_model_train_1084.pkl'\n\n# Chargement du modèle sauvegardé\nwith open(model_filename, 'rb') as file:\n    loaded_model = pickle.load(file)\n\n# Utilisation du modèle chargé pour faire des prédictions ou d'autres analyses\nprint(loaded_model.summary())\n\n# Supposons que nous avons un nouveau jeu de données X_new pour lequel on souhaite prédire y_log\nX_new = sm.add_constant(X_new)  # Ajout de l'intercept si\n\n# Faire des prédictions\ny_pred_log = loaded_model.predict(X_new)\n\n# Si vous souhaitez obtenir les prédictions en échelle normale (non log), vous pouvez utiliser np.exp\ny_pred = np.exp(y_pred_log)\n\n"

###### Nous allons créer une dataframe  qui contient des éléments utiles au calcul du taux de pauvreté

In [26]:
import pandas as pd
import numpy as np

# Faire les prédictions avec le modèle chargé
y_pred_OLS_log_sans_pca = results.predict(X_train_1084)

# Créer la DataFrame avec les valeurs souhaitées, y compris la colonne 'milieu'
df_results = pd.DataFrame({
    'y_pred_OLS_log_sans_pca': y_pred_OLS_log_sans_pca,
    'log_y_actual': y_log,  # log(y) réel
    'y_actual': y,  # y réel
    'milieu': final_df['milieu'],  # Ajouter la colonne 'milieu' correspondante
    'taille_menage': final_df['hhsize'],
    'poids_menage': final_df['hhweight']
})

# Afficher les premières lignes de la DataFrame pour vérification
print(df_results.head())


   y_pred_OLS_log_sans_pca  log_y_actual   y_actual  milieu  taille_menage  \
0                12.696511     12.420373  247799.03  Urbain              6   
1                12.619463     12.420373  247799.03  Urbain              6   
2                12.796533     12.420373  247799.03  Urbain              6   
3                12.528940     12.420373  247799.03  Urbain              6   
4                12.682622     12.420373  247799.03  Urbain              6   

   poids_menage  
0      547.8328  
1      547.8328  
2      547.8328  
3      547.8328  
4      547.8328  


###### Nous scindons cette dataframe en deux une qui contient les prédictions uniquement en zone rurale et l'autre qui contient les les prédictions uniquement en urbain

In [27]:
# Créer la DataFrame df_results_urbain avec les valeurs où 'milieu' est 'Urbain'
df_results_urbain = df_results[df_results['milieu'] == 'Urbain']

# Créer la DataFrame df_results_rural avec les valeurs où 'milieu' est 'Rural'
df_results_rural = df_results[df_results['milieu'] == 'Rural']

# Afficher les premières lignes des deux DataFrames pour vérification
print("DataFrame Urbain:")
print(df_results_urbain.head())

print("\nDataFrame Rural:")
print(df_results_rural.head())


DataFrame Urbain:
   y_pred_OLS_log_sans_pca  log_y_actual   y_actual  milieu  taille_menage  \
0                12.696511     12.420373  247799.03  Urbain              6   
1                12.619463     12.420373  247799.03  Urbain              6   
2                12.796533     12.420373  247799.03  Urbain              6   
3                12.528940     12.420373  247799.03  Urbain              6   
4                12.682622     12.420373  247799.03  Urbain              6   

   poids_menage  
0      547.8328  
1      547.8328  
2      547.8328  
3      547.8328  
4      547.8328  

DataFrame Rural:
     y_pred_OLS_log_sans_pca  log_y_actual   y_actual milieu  taille_menage  \
100                13.186958     12.787775  357816.44  Rural              4   
101                12.673123     12.787775  357816.44  Rural              4   
102                12.480489     12.787775  357816.44  Rural              4   
103                12.853891     12.787775  357816.44  Rural           

##### Calcul du taux de pauvreté en zone rurale et urbaine

In [29]:
import numpy as np

# Définir le seuil en utilisant la valeur log(345520)
seuil = np.log(345520)

# Ajouter les colonnes binaires en fonction des seuils dans df_results
df_results['y_log_binaire'] = (df_results['log_y_actual'] < seuil).astype(int)
df_results['y_pred_OLS_log_sans_pca_binaire'] = (df_results['y_pred_OLS_log_sans_pca'] < seuil).astype(int)

# Ajouter les colonnes binaires pour les DataFrames urbain et rural en utilisant .loc[] pour éviter l'avertissement
df_results_urbain.loc[:, 'y_pred_OLS_log_sans_pca_binaire'] = (df_results_urbain['y_pred_OLS_log_sans_pca'] < seuil).astype(int)
df_results_rural.loc[:, 'y_pred_OLS_log_sans_pca_binaire'] = (df_results_rural['y_pred_OLS_log_sans_pca'] < seuil).astype(int)

# Calcul du total des ménages pondérés pour chaque groupe
total_menage = (df_results['taille_menage'] * df_results['poids_menage']).sum()
total_menage_urbain = (df_results_urbain['taille_menage'] * df_results_urbain['poids_menage']).sum()
total_menage_rural = (df_results_rural['taille_menage'] * df_results_rural['poids_menage']).sum()

# Calcul des taux de pauvreté
taux_pauvrete_reel_echantillon = (df_results['taille_menage'] * df_results['poids_menage'] * df_results['y_log_binaire']).sum() / total_menage
taux_pauvrete_OLS_log = (df_results['taille_menage'] * df_results['poids_menage'] * df_results['y_pred_OLS_log_sans_pca_binaire']).sum() / total_menage
taux_pauvrete_urbain_OLS_log = (df_results_urbain['taille_menage'] * df_results_urbain['poids_menage'] * df_results_urbain['y_pred_OLS_log_sans_pca_binaire']).sum() / total_menage_urbain
taux_pauvrete_rural_OLS_log = (df_results_rural['taille_menage'] * df_results_rural['poids_menage'] * df_results_rural['y_pred_OLS_log_sans_pca_binaire']).sum() / total_menage_rural

# Affichage des résultats
print(f"Taux de pauvreté réel (échantillon) : {taux_pauvrete_reel_echantillon:.2%}")
print(f"Taux de pauvreté prédit par OLS (log consommation par tête, sans PCA) : {taux_pauvrete_OLS_log:.2%}")
print(f"Taux de pauvreté en zone urbaine prédit par OLS : {taux_pauvrete_urbain_OLS_log:.2%}")
print(f"Taux de pauvreté en zone rurale prédit par OLS : {taux_pauvrete_rural_OLS_log:.2%}")


Taux de pauvreté réel (échantillon) : 39.30%
Taux de pauvreté prédit par OLS (log consommation par tête, sans PCA) : 29.43%
Taux de pauvreté en zone urbaine prédit par OLS : 16.32%
Taux de pauvreté en zone rurale prédit par OLS : 42.44%
